# Identyfikacja klientów o wysokim ryzyku odejścia do konkurencji (CHURN)


Jesteś analitykiem firmy telekomunikacyjnej. Twoim zadaniem jest opracowanie modelu identyfikującego klientów o wysokim ryzyku odejścia do konkurencji (CHURN) oraz ocena efektywności ekonomicznej jego wdrożenia.

Aktualna sytuacja jest następująca:

1. Na jednym kliencie mamy 700 USD marży.
2. Klientowi, któremu kończy się niebawem umowa, oferujemy 100 USD dolarów zachęty (bonus), by z nami został. Nie wykorzystujemy w tym celu żadnego modelu.
3. Koszt nawiązania takiego kontaktu (praca telemarketingu) wynosi 50 USD
4. Nie każdy z klientów, do którego zadzwonimy, decyduje się na przedłużenie umowy: w takiej sytuacji ponosimy koszt 50 USD, nie wydajemy jednak 100 USD na bonus.

Z działu sprzedaży otrzymałeś plik z następującymi danymi:

**State**: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ

**Area Code**: the three-digit area code of the corresponding customer’s phone number

**Phone**: the remaining seven-digit phone number

**Account Length**: the number of days that this account has been active

**Int’l Plan**: whether the customer has an international calling plan

**VMail Plan**: whether the customer has a voice mail feature

**VMail Message**: presumably the average number of voice mail messages per month

**Day Mins**: the total number of calling minutes used during the day

**Day Calls**: the total number of calls placed during the day

**Day Charge**: the billed cost of daytime calls

**Eve Mins, Eve Calls, Eve Charge**: the billed time, # of calls and cost for calls placed during the evening

**Night Mins, Night Calls, Night Charge**: the billed time, # of calls and cost for calls placed during nighttime

**Intl Mins, Intl Calls, Intl Charge**: the billed time, # of calls and cost for international calls

**CustServ Calls**: the number of calls placed to Customer Service

**Churn?**: whether the customer left. 0: stayed (no churn), 1: left (churn)

Powodzenia!
---

Uwaga 1: tutorial ten jest adaptacją tutoriala dostępnego tutaj: https://www.pycaret.org/tutorials/html/CLF101.html 

Uwaga 2: dane źródłowe są dostępne np. tutaj://sagemaker-examples.readthedocs.io/en/latest/sagemaker_neo_compilation_jobs/xgboost_customer_churn/xgboost_customer_churn_neo.html

## Pozyskanie danych

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('customers_churn.csv', index_col=False)

In [ ]:
dataset.head()

In [ ]:
# sprawdźmy kształt danych
dataset.shape

In [ ]:
# wyodrębnijmy zbiór testujący (dane te wykorzystamy dopiero na końcu do ewaluacji ostatecznego modelu)
data = dataset.sample(frac = 0.8)
unseen_data = dataset.drop(data.index)

In [ ]:
data.shape, unseen_data.shape

In [ ]:
data.tail()

## Konfiguracja środowiska PyCaret

Funkcja `setup()` inicjalizuje środowisko w pycaret i tworzy sekwencję transformacji przygotowania danych do modelowania i uruchomienia.

`setup()` musi być wywołana przed wykonaniem jakiejkolwiek innej funkcji w pycaret. Pobiera ona dwa obowiązkowe parametry: ramkę danych pandas i nazwę kolumny docelowej. Wszystkie inne parametry są opcjonalne i używane do dostosowania potoku przetwarzania wstępnego.

Podczas wykonywania `setup()`, algorytm PyCaret automatycznie identyfikuje typy danych dla wszystkich cech na podstawie ich właściwości. Nie zawsze jednak typ danych jest zidentyfikowany poprawnie. Aby to uwzględnić, PyCaret wyświetla tabelę zawierającą cechy i ich typy danych po wykonaniu setup(). Jeśli wszystkie typy danych są poprawnie zidentyfikowane można nacisnąć `enter` aby kontynuować lub `quit` aby zakończyć eksperyment. 

Upewnienie się, że typy danych są poprawne ma fundamentalne znaczenie, ponieważ są one podstawą dla trenowania i ewaluacji modeli w PyCaret. Zadania te są wykonywane inaczej dla każdego typu danych, dlatego bardzo ważna jest poprawna identyfikacja typów zmiennych.

In [ ]:
from pycaret.classification import *

In [ ]:
exp_DKU_1 = setup(data = data, target = 'churn', train_size=0.8, imputation_type='iterative', ignore_features = ['phone'], session_id=123) 

Po pomyślnym uruchomieniu `setup` wyświetla tabelę z kluczowymi informacjami. Większość z nich jest związana z potokiem wstępnego przetwarzania, z których najbardziej istotne są:

**session_id** : Pseudolosowa liczba rozprowadzana jako ziarno we wszystkich funkcjach dla późniejszej odtwarzalności. Jeśli nie zostanie podany żaden `session_id`, automatycznie generowana jest liczba losowa, która jest dystrybuowana do wszystkich funkcji. W tym eksperymencie, `session_id` jest ustawiony jako 123 dla późniejszej odtwarzalności.

**Target Type** : Binarny lub Wieloklasowy. Typ celu jest automatycznie wykrywany i pokazywany. Nie ma różnicy w sposobie wykonywania eksperymentu dla problemów Binarnych i Wieloklasowych. Wszystkie funkcje są identyczne.

**Label Encoded** : Kiedy zmienna docelowa jest typu string (np. 'Tak' lub 'Nie') zamiast 1 lub 0, PyCaret automatycznie koduje etykietę na 1 i 0 i wyświetla mapowanie (0 : Nie, 1 : Tak). 

**Original Data** : Wyświetla oryginalny kształt zbioru danych. W tym eksperymencie (2666, 23) oznacza 2666 próbek i 23 cechy, w tym kolumnę docelową.

**Missing Values** : Jeśli w oryginalnych danych brakuje wartości, zostanie to wyświetlone jako `True`. Dla tego eksperymentu nie ma brakujących wartości w zbiorze danych.

**Numeric Features** : Liczba cech zidentyfikowanych jako numeryczne. W tym zestawie danych, 15 z 21 cech jest wnioskowanych jako numeryczne.

**Categorical Features** : Liczba cech zidentyfikawanych jako kategoryczne. W tym zbiorze danych, 6 z 21 cech jest określanych jako kategoryczne.

**Transformed Train Set** : Wyświetla kształt przekształconego zbioru treningowego. Zauważ, że oryginalny kształt (2666, 23) jest przekształcony do (2132, 96) dla przekształconego zbioru treningowego, a liczba cech wzrosła do 96 z 23 z powodu kodowania.

**Transformed Test Set** : Wyświetla kształt przekształconego zbioru testowego/hold-out. W zbiorze testowym znajduje się 534 próbek. Podział ten przeprowadzony jest w domyślnej proporcji 70/30, która może być zmieniona przy pomocy parametru `train_size` w `setup`ie.

Zauważ, że kilka zadań, które są niezbędne do przeprowadzenia modelowania, jest realizowanych automatycznie, np. imputacja brakujących wartości (w tym przypadku nie ma brakujących wartości w danych treningowych, ale nadal potrzebujemy imputatorów dla niewidzianych danych), kodowanie kategorialne itp. Większość parametrów w setup() jest opcjonalna i używana do dostosowywania potoku wstępnego przetwarzania. 

## Eksploracyjna analiza danych (EDA)

Eksploracyjna analiza danych możliwa jest w PyCaret dzięki funkcji `eda()`

In [ ]:
eda()

Do Eksploracyjnej Analizy Danych (EDA) możemy też użyć modułu Pandas Profiling. Więcej szczegółów można znaleźć na stronie: https://github.com/pandas-profiling/pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(dataset, title="Pandas Profiling Report")

In [ ]:
profile.to_notebook_iframe()

## Trening i porównanie modeli

Na początku procesu modelowania zalecane jest porównanie najprostszych wersji wszystkich modeli bazowych w celu oceny ich jakości (chyba że dokładnie wiesz, jakiego modelu potrzebujesz). 

Funkcja `compare_models` trenuje wszystkie modele z biblioteki i ocenia je przy użyciu walidacji krzyżowej. 

Efektem jest tabela wyników, która pokazuje średnią Accuracy, AUC, Recall, Precyzję, F1, Kappa i MCC dla wszystkich warstw (domyślnie 10) wraz z czasami treningu.

In [ ]:
best_model = compare_models(sort = 'Recall')

Uruchomienie jednej komendy umożliwiło trening i ewaluację ponad 15 modeli. Tabela wyników wyświetlona powyżej zaznacza najwyższe wyniki metryki tylko dla celów porównawczych. Tabela domyślnie posortowana jest wg 'Accuracy' (od najwyższej do najniższej), ale można to zmienić poprzez podanie parametru sortowania. Przykładowo: `compare_models(sort = 'AUC')` posortuje siatkę według AUC zamiast Accuracy. 

Jeśli chcesz zmienić parametr `fold` z domyślnej wartości 10 na inną wartość, możesz użyć parametru `fold`. Przykładowo, `compare_models(fold = 5)` porówna wszystkie modele w procesie 5-krotnej walidacji krzyżowej. Zmniejszenie liczby warstw poprawi czas treningu. Domyślnie, `compare_models` zwraca najlepiej działający model w oparciu o domyślną kolejność sortowania, ale może być użyty do zwrócenia listy N najlepszych modeli poprzez użycie parametru `n_select`.

In [ ]:
print(best_model)

## Tworzenie i optymalizacja najlepszego modelu

### Stworzenie modelu
`Create_model` jest najbardziej szczegółową funkcją w PyCaret i często podstawą większości funkcjonalności PyCaret. Jak sama nazwa wskazuje, funkcja ta trenuje i ocenia model używając walidacji krzyżowej, którą można skonfigurować za pomocą parametru `fold`. Na wyjściu wypisuje tabelę wyników, która pokazuje Accuracy, AUC, Recall, Precision, F1, Kappa i MCC w zależności od liczby warstw.

W pozostałej części tego poradnika będziemy pracować z modelem `dt` (Decision Tree Classifier): praktycznie zwycięskim w kategorii Recall, niemniej z interesującym potencjałym udoskonalenia.

W bibliotece modeli PyCaret dostępnych jest 18 modeli klasyfikacyjnych. Aby zobaczyć listę wszystkich klasyfikatorów można użyć funkcji `models`.

In [ ]:
models()

In [ ]:
dt = create_model('dt')

Zauważ, że średni wynik wszystkich modeli zgadza się z wynikiem wypisanym w `compare_models()`. Dzieje się tak dlatego, że metryka wypisana w siatce wyników `compare_models()` jest średnią wyników dla wszystkich warstw walidacji krzyżowej (CV). 

Podobnie jak w przypadku `compare_models()`, jeśli chcesz zmienić parametr `fold` z domyślnej wartości 10 na inną wartość, możesz użyć parametru `fold`. Przykładowo, `create_model('dt', fold = 5)` utworzy klasyfikator drzewa decyzyjnego używając 5-krotnego warstwowego CV.

## Interaktywna analiza modelu

PyCaret umożliwia kompleksową, interaktywną analizę wytrenowanego modelu, z wykorzystaniem funkcji `dashboard()`

In [ ]:
dashboard(dt)

Funkcja `check_fairness` umożliwia sprawdzenie, czy w wyniku modelowania któraś z grup nie jest dyskryminowana. Pozwala porównać pod tym kątem różne sub-populacje. Porównaj: https://github.com/fairlearn/fairlearn

### Udoskonalenie modelu

Tworząc model przy użyciu funkcji `create_model()`, PyCaret używa domyślnych hiperparametrów do trenowania modelu. W celu dostrojenia hiperparametrów używana jest funkcja `tune_model()`. Funkcja ta automatycznie dostraja hiperparametry modelu używając `Random Grid Search` na wcześniej zdefiniowanej przestrzeni poszukiwań. 

Na wyjściu wypisywana jest tabela wyników, która pokazuje Accuracy, AUC, Recall, Precyzję, F1, Kappa i MCC dla najlepszego modelu. Aby użyć niestandardowej siatki wyszukiwania, możesz przekazać parametr `custom_grid` w funkcji `tune_model`.

In [ ]:
tuned_dt = tune_model(dt, optimize='Recall')

Domyślnie, `tune_model` optymalizuje Accuract, ale można to zmienić używając parametru `optimize`. Na przykład: `tune_model(dt, optimize = 'AUC')` będzie szukał takich hiperparametrów klasyfikatora drzewa decyzyjnego, które dają najwyższy wynik AUC zamiast Accuracy. W naszym przykładzie wykorzystaliśmy Recall ze względu na specyfikę problemu (analiza CHURN).

Generalnie, gdy zbiór danych jest niezrównoważony (jak np. zbiór danych kredytowych, z którym pracujemy), Accuracy nie jest dobrą metryką do rozważenia. Metodologia stojąca za wyborem właściwej metryki do oceny klasyfikatora wykracza poza zakres tego tutoriala, ale jeśli chcesz dowiedzieć się więcej na ten temat, możesz spojrzeć np. tutaj (https://medium.com/@MohammedS/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b).

## Ewaluacja modelu

### Wykresy

Przed finalizacją modelu, warto wykorzystać funkcję `plot_model()` do analizy wydajności w różnych wymiarach, takich jak AUC, confusion_matrix, granica decyzji itp.

Dostępnych jest 15 różnych wykresów - aby zobaczyć ich listę, sprawdź dokumentację `plot_model()`.

#### Macierz pomyłek (Confusion Matrix)

In [ ]:
%matplotlib inline
plot_model(tuned_dt, plot = 'confusion_matrix')

#### AUC Plot

In [ ]:
plot_model(tuned_dt, plot = 'auc')

#### Precision-Recall Curve

In [ ]:
plot_model(tuned_dt, plot = 'pr')

#### Feature Importance Plot

In [ ]:
plot_model(tuned_dt, plot = 'feature')

Innym sposobem na analizę wydajności modeli jest użycie funkcji `evaluate_model()`, która wyświetla interfejs użytkownika dla wszystkich dostępnych wykresów dla danego modelu. 

In [ ]:
evaluate_model(tuned_dt)

### Optymalizacja progu (threshold)

PyCaret pozwala też na optymalizację progu w klasyfikacji binarnej.

In [ ]:
optimize_threshold(dt)

## Predykcja na zbiorze testowym/hold-out

Przed sfinalizowaniem modelu, zalecane jest wykonanie testu efektywności modelu na zbiorze testowym/hold-out.

Jeśli spojrzysz na tabelę wyników w części `setup` powyżej, zobaczysz, że 20% (534 próbki) danych zostało wyodrębnione jako próbka testowa/hold-out. Wszystkie metryki oceny, które widzieliśmy powyżej to wyniki walidacji krzyżowej oparte tylko na zestawie treningowym (80%). 

W tym momencie, używając naszego ostatecznego wytrenowanego modelu przechowywanego w zmiennej `tuned_dt`, sprawdzimy wyniki predykcji (i ich metryki) na zbiorze testowym i zobaczymy, na ile różnią się one od wyników walidacji krzyżowej w procesie trenowania.

In [ ]:
predict_model(tuned_dt)

Jak widać, w procesie trenowania otrzymaliśmy Recall na poziomie 0.8076, zaś na zbiorze testowym nieco lepszy wynik: 0.8485.

## Udostępnienie modelu

Ostatnim etapem naszego eksperymentu jest finalizacja modelu.

Typowy proces uczenia maszynowego w PyCaret zaczyna się od `setup()`, po którym następuje porównanie wszystkich modeli przy użyciu `compare_models()` i wybranie kilku modeli najlepiej rokujących (w oparciu o interesującą nas metrykę), a następnie ich udoskonalenie np. poprzez poprzez optymalizację hiperparametrów, ensembling, stacking itp. 

Taki proces ostatecznie prowadzi do najlepszego modelu, który możesz wykorzystać w środowisku produkcyjnym do do przewidywań na nowych, nieznanych danych. 

Funkcja `finalize_model()` dopasowuje model do całego zbioru danych, włączając w to próbkę testową (w tym przypadku 30%).

Jej celem jest wytrenowanie modelu na kompletnym zbiorze danych, zanim zostanie on wdrożony do produkcji.

In [ ]:
final_dt = finalize_model(tuned_dt)

In [ ]:
print(final_dt)

Uwaga
___
Podczas finalizacji z wykorzystaniem `finalize_model()` do trenowania modelu wykorzystywany jest cały zbiór danych (czyli treningowy i testowy).

W efekcie, sprawdzanie tego modelu na zbiorze testowym/hold'out nie ma sensu (jest mylące): dane były dostępne podczas trenowania. 

W przykładzie poniżej wykorzystamy funkcję `predict_model` do oceny modelu zfinalizowanego `final_dt`. Jak się domyślasz, wyniki oceny będzie lepszy niż poprzednio.

In [ ]:
predict_model(final_dt)

Zwróć uwagę, że `Recall` w `final_dt` wzrósł z 0.8485 do poziomu 0.8636 mimo tego, że model jest taki sam. Dzieje się tak, ponieważ model `final_dt` został wytrenowany na pełnym zbiorze danych, w tym na zbiorze testowym/hold-out. **Jeszcze raz: pamiętaj, że operacja ta daje mylące wyniki!**

### Predykcja nieznanych danych

Funkcja `predict_model()` może być wykorzystana do predykcji na nieznanym zbiorze danych. Jedyną różnicą w stosunku do czynności realizowanych powyżej jest to, że tym razem przekażemy parametr `unseen_data` z linii 7. 

Parametr `unseen_data` jest zmienną utworzoną na początku tutoriala i zawiera 5% (1200 próbek) oryginalnego zbioru danych, który nigdy nie był udostępniony PyCaret na potrzeby trenowania.

In [ ]:
unseen_predictions = predict_model(final_dt, data=unseen_data)
unseen_predictions.head()

Efektem uruchomienia `predict_model` jest dodanie do zbioru `unseen_data` kolumn `Label` i `Score` zawierających odpowiednio predykcję i jej prawdopodobieństwo. 

Zauważ, że przewidywane wyniki są stosowane do oryginalnego zbioru danych, podczas gdy wszystkie transformacje danych są automatycznie wykonywane w tle. 

Mając te kolumny możesz też sprawdzić metryki jakości modelu korzystając z modułu `pycaret.utils`:

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['churn'], unseen_predictions['Label'], metric = 'Recall')

In [ ]:
check_metric(unseen_predictions['churn'], unseen_predictions['Label'], metric = 'Accuracy')

### Zapis modelu

Finalizacja modelu `tuned_dt`, zapisana w `final_dt`, kończy proces modelowania.

Pojawia się jednak pytanie: w jaki sposób zrobić predykcję na zupełnie nowych danych, np. w systemie produkcyjnym? W szczególności, czy będziemy musieli ponownie poddawać te dane transformacjom takim, jak w procesie modelowania?

Odpowiedź brzmi nie: wbudowana w PyCaret funkcja `save_model()` pozwala na zapisanie modelu wraz z całym potokiem transformacji do późniejszego wykorzystania na produkcji. Poniżej zapiszemy nasz cały potok do pliku `Final Decision Tree Model Dec2021`.

In [ ]:
save_model(final_dt,'Final Decision Tree Model Jan2022')

### Wczytanie zapisanego modelu

Aby załadować zapisany model np. na innym komputerze lub środowisku produkcyjnym, użyjemy funkcji `load_model()`. Następnie w prosty sposób zastosowalibyśmy zapisany model na nowych danych.

In [ ]:
saved_final_dt = load_model('Final Decision Tree Model Jan2022')

Tak wczytany model możesz po prostu użyć do przewidywania na nowych danych używając tej samej funkcji `predict_model()`. Poniżej zastosujemy go do przewidywania tych samych `data_unseen`, których użyliśmy w sekcji powyżej.

In [ ]:
unknowns = pd.read_csv('customers_churn.csv')

In [ ]:
new_prediction = predict_model(saved_final_dt, data=unknowns)

In [ ]:
new_prediction.head()

# Stworzenie aplikacji

PyCaret umożliwia uruchomienie prostej aplikacji serwującej model. 

In [ ]:
create_app(saved_final_dt)